## Read the script

In [1]:
import os
import numpy as np
import mne
from ieeg.io import get_data, raw_from_layout, save_derivative
from ieeg.navigate import trial_ieeg, outliers_to_nan, channel_outlier_marker, crop_empty_data
from ieeg.calc.scaling import rescale
from ieeg.viz.ensemble import chan_grid
from ieeg.timefreq.utils import crop_pad, wavelet_scaleogram
from ieeg.viz.parula import parula_map
save_dir='C:\\Users\\bl314\\Box\\CoganLab\\IndividualMeetings\\Baishen\\ieeg_results\\lexical_delay'

In [4]:
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab")
layout = get_data("LexicalDecRepDelay", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")
subject = subjects[4]

if subject=='D0054':
    subject_Tag = 'D54'
elif subject=='D0055':
    subject_Tag = 'D55'
elif subject=='D0070':
    subject_Tag = 'D70'
elif subject=='D0070':
    subject_Tag = 'D70'
elif subject=='D0094':
    subject_Tag = 'D94'
elif subject=='D0101':
    subject_Tag = 'D101'
elif subject=='D0102':
    subject_Tag = 'D102'
elif subject=='D0103':
    subject_Tag = 'D103'
elif subject=='D0107':
    subject_Tag = 'D107B'
else:
    print("Subject not found, please check.")

## Plot subj

In [5]:
from ieeg.viz.mri import plot_subj

# plot the subject brain
fig1 = plot_subj(subject_Tag)
#Label every electrode
mne.viz.set_3d_view(fig1, azimuth=150, elevation=70, focalpoint="auto",
                    distance="auto")

C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\viz\mri.py:696: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


Using pyvistaqt 3d backend.
Could not estimate rigid Talairach alignment, using identity matrix


## Load subj
### <span style="color:red">Please make sure that the **event.tsv** files in the **clean** derivatives do not contain **"BAD boundary"** lines, or remove them if they have</span>

In [6]:
raw = raw_from_layout(layout.derivatives['derivatives/clean'], subject=subject, desc='clean',extension='.edf',preload=True)

Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0055\ieeg\sub-D0055_task-LexicalDecRepDelay_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0055\ieeg\sub-D0055_task-LexicalDecRepDelay_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0055\ieeg\sub-D0055_task-LexicalDecRepDelay_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0055\ieeg\sub-D0055_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\s

C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0055\ieeg\sub-D0055_task-LexicalDecRepDelay_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0055\ieeg\sub-D0055_task-LexicalDecRepDelay_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0055\ieeg\sub-D0055_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0055\ieeg\sub-D0055_task-LexicalDecRepDelay_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 424 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0055\ieeg\sub-D0055_task-LexicalDecRepDelay_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0055\ieeg\sub-D0055_task-LexicalDecRepDelay_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0055\ieeg\sub-D0055_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0055\ieeg\sub-D0055_task-LexicalDecRepDelay_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 424 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0055\ieeg\sub-D0055_task-LexicalDecRepDelay_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0055\ieeg\sub-D0055_task-LexicalDecRepDelay_acq-01_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0055\ieeg\sub-D0055_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 422 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading 0 ... 7540735  =      0.000 ...  3682.000 secs...


## Remove eeg channels
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms.html
### <span style="color:red">Please pay attention to the section above and see what channels do not have locations when the raw was loaded. Write them down and remove them. They are usually EEG channels</span>

In [7]:
# Remove EEG channels for D101
found=1
if subject=='D0053':
    eeg_channels_to_exclude=[]
elif subject=='D0055':
    eeg_channels_to_exclude=[]
elif subject=='D0054':
    eeg_channels_to_exclude=[]
elif subject=='D0070':
    eeg_channels_to_exclude=[]
elif subject=='D0094':
    eeg_channels_to_exclude=[]
elif subject=='D0101':
    eeg_channels_to_exclude = ['Fp1', 'Fp2', 'F3','F4','C3','C4','P3','P4','F7','F8','T3','T4','T5','T6','O1','O2','Fz','Cz','Pz']
elif subject=='D0102':
    eeg_channels_to_exclude = ['T5', 'T6', 'FZ', 'CZ', 'PZ', 'FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', '02', 'F7', 'F8', 'T3', 'T4']
elif subject=='D0103':
    eeg_channels_to_exclude = ['FZ', 'CZ', 'PZ', 'F7', 'F8', 'T5', 'T6', 'FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'T3', 'T4']
elif subject=='D0107':
    eeg_channels_to_exclude=[]
else:
    print("Subject not found, please check.")
    found=0
    
if found and eeg_channels_to_exclude:
    raw.drop_channels(eeg_channels_to_exclude)
#raw.load_data()

## Remove outlier channels

In [8]:
# mark channel outliers as bad
raw.info['bads'] = channel_outlier_marker(raw, 3, 2)
# Exclude bad channels
raw.drop_channels(raw.info['bads'])
#raw.load_data()

outlier round 1 channels: ['RASF7']
outlier round 1 channels: ['RASF7', 'RPI14']
outlier round 1 channels: ['RASF7', 'RPI14', 'RPMT11']
outlier round 1 channels: ['RASF7', 'RPI14', 'RPMT11', 'LMMT13']
outlier round 2 channels: ['RASF7', 'RPI14', 'RPMT11', 'LMMT13', 'RPI6']
outlier round 2 channels: ['RASF7', 'RPI14', 'RPMT11', 'LMMT13', 'RPI6', 'RAMT12']
outlier round 2 channels: ['RASF7', 'RPI14', 'RPMT11', 'LMMT13', 'RPI6', 'RAMT12', 'RPMT12']
outlier round 2 channels: ['RASF7', 'RPI14', 'RPMT11', 'LMMT13', 'RPI6', 'RAMT12', 'RPMT12', 'RPIT8']
outlier round 2 channels: ['RASF7', 'RPI14', 'RPMT11', 'LMMT13', 'RPI6', 'RAMT12', 'RPMT12', 'RPIT8', 'LMMT12']


<RawEDF | sub-D0055_task-LexicalDecRepDelay_acq-01_run-01_desc-clean_ieeg.edf, 182 x 7540736 (3682.0 s), ~10.23 GB, data loaded>

## Wavelet spectrogram

In [9]:
if not os.path.exists(os.path.join(save_dir, subject)):
    os.mkdir(os.path.join(save_dir, subject)) 
if not os.path.exists(os.path.join(save_dir, subject,'wavelet')):
    os.mkdir(os.path.join(save_dir, subject,'wavelet'))
if not os.path.exists(os.path.join(save_dir, subject,'multitaper')):
    os.mkdir(os.path.join(save_dir, subject,'multitaper'))

In [10]:
# Wavelet is good to detect and remove muscle artifact channels
# Also plot the subject brain
for task, task_Tag in zip(('Repeat','Yes_No'),('Rep','YN')):
    for epoch, t, tag in zip(
        ('Auditory_stim/'+task+'/CORRECT','Delay/'+task+'/CORRECT','Resp/'+task+'/CORRECT'),
        ((-0.5, 1.5),(-0.5, 1.5),(-0.5, 1)),
        ('Auditory-'+task_Tag,'Delay-'+task_Tag,'Resp-'+task_Tag)
    ):
        
        # Get the spectras
        t1 = t[0] - 0.5
        t2 = t[1] + 0.5
        times = (t1, t2)
        trials = trial_ieeg(raw, epoch, times, preload=True)
        outliers_to_nan(trials, outliers=10)
        
        ##############################
        ####### Wavelet ##############
        ##############################
        
        spectra_wavelet = wavelet_scaleogram(trials, n_jobs=-3, decim=int(
            raw.info['sfreq'] / 200)) # 1/10 of the timepionts, don't take too long
        crop_pad(spectra_wavelet, "0.5s") # cut the first and final 0.5s, change to zero
        
        # Get the baseline
        if epoch=='Auditory_stim/'+task+'/CORRECT':
            base_wavelet = spectra_wavelet.copy().crop(-0.5, 0)
            base_wavelet = base_wavelet.average(lambda x: np.nanmean(x, axis=0), copy=True)
        
        # Baseline correction
        spectra_wavelet = spectra_wavelet.average(lambda x: np.nanmean(x, axis=0), copy=True)   
        spectra_wavelet = rescale(spectra_wavelet, base_wavelet, copy=True, mode='ratio')
        spectra_wavelet._data = np.log10(spectra_wavelet._data) * 20
    
        # Save spectras
        filename = os.path.join(save_dir, subject,'wavelet',f'{tag}-tfr.h5')
        mne.time_frequency.write_tfrs(filename, spectra_wavelet, overwrite=True)
        
        # Make spectrograms
        chan_grids=chan_grid(spectra_wavelet, size = (20,10),vlim=(-2, 2), cmap=parula_map)
    
        # Save spectrograms
        fig_count=0
        for fig in chan_grids:
            figdir = os.path.join(save_dir, subject,'wavelet',f'{tag}_{fig_count+1}.jpg')
            chan_grids[fig_count].savefig(figdir,dpi=300)   
            fig_count+=1
            
        # Clean memory
        del spectra_wavelet,filename
        
        ##############################
        ####### Multitapper ##########
        ##############################
        
        freq = np.linspace(0.5, 200, num=80)
        kwargs = dict(average=False, n_jobs=-3, freqs=freq, return_itc=False,
                      n_cycles=freq/2, time_bandwidth=4,
                      # n_fft=int(trials.info['sfreq'] * 2.75),
                      decim=20, )
                      # adaptive=True)
        spectra_multitaper = trials.compute_tfr(method="multitaper",  **kwargs)
        crop_pad(spectra_multitaper, "0.5s") # cut the first and final 0.5s, change to zero
        
                # Get the baseline
        if epoch=='Auditory_stim/'+task+'/CORRECT':
            base_multitaper = spectra_multitaper.copy().crop(-0.5, 0)
            base_multitaper = base_multitaper.average(lambda x: np.nanmean(x, axis=0), copy=True)
 
        # Baseline correction
        spectra_multitaper = spectra_multitaper.average(lambda x: np.nanmean(x, axis=0), copy=True)   
        spectra_multitaper = rescale(spectra_multitaper, base_multitaper, copy=True, mode='ratio')
        
        # Save spectras
        filename = os.path.join(save_dir, subject,'multitaper',f'{tag}-tfr.h5')
        mne.time_frequency.write_tfrs(filename, spectra_multitaper, overwrite=True)
        
        # Make spectrograms
        chan_grids=chan_grid(spectra_multitaper, size = (20,10),vlim=(0.7, 1.4), cmap=parula_map)
        
        # Save spectrograms
        fig_count=0
        for fig in chan_grids:
            figdir = os.path.join(save_dir, subject,'multitaper',f'{tag}_{fig_count+1}.jpg')
            chan_grids[fig_count].savefig(figdir,dpi=300)   
            fig_count+=1
        
        del trials,spectra_multitaper,filename
    del base_wavelet,base_multitaper

Used Annotations descriptions: [np.str_('Auditory_stim/Repeat/Nonword/banic/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/berin/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/boril/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/caris/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/cazel/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/delin/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/delin/ERR_TASK/REP_YN'), np.str_('Auditory_stim/Repeat/Nonword/devim/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/galed/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/galel/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/gapel/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/gorel/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/havel/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/herib/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/janem/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/japel/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/jokel/CORRECT'), np.str_('Auditory_stim/Re

[Parallel(n_jobs=-3)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=-3)]: Done   1 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-3)]: Done  12 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-3)]: Done  25 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-3)]: Done  38 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-3)]: Done  53 tasks      | elapsed:   16.1s
[Parallel(n_jobs=-3)]: Done  68 tasks      | elapsed:   19.2s
[Parallel(n_jobs=-3)]: Done  85 tasks      | elapsed:   23.1s
[Parallel(n_jobs=-3)]: Done 102 tasks      | elapsed:   27.4s
[Parallel(n_jobs=-3)]: Done 121 tasks      | elapsed:   30.2s
[Parallel(n_jobs=-3)]: Done 141 out of 181 | elapsed:   35.5s remaining:   10.0s
[Parallel(n_jobs=-3)]: Done 160 out of 181 | elapsed:   37.9s remaining:    4.9s
[Parallel(n_jobs=-3)]: Done 179 out of 181 | elapsed:   40.4s remaining:    0.4s
[Parallel(n_jobs=-3)]: Done 181 out of 181 | elapsed:   40.4s finished
C:\Users\bl314\AppData\Local\Temp\ipykernel_79

Applying baseline correction (mode: ratio)


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\calc\scaling.py:67: RuntimeWarning: invalid value encountered in divide
  d /= m


No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No basel

...

Applying baseline correction (mode: ratio)


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\calc\scaling.py:67: RuntimeWarning: invalid value encountered in divide
  d /= m


No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No basel

[Parallel(n_jobs=-3)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=-3)]: Done  42 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-3)]: Done 132 tasks      | elapsed:   30.9s
[Parallel(n_jobs=-3)]: Done 181 out of 181 | elapsed:   38.8s finished


Applying baseline correction (mode: ratio)
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction appl